## Calculating Sharpe and Sortino Ratios

In [61]:
"""
Sharpe + Sortino (step-by-step, readable)

What this script does:
1) Loads price data (CSV) for one stock (Date, Close)
2) Computes periodic returns (weekly by default)
3) Computes annualized volatility (total + downside)
4) Computes Sharpe + Sortino using a forward-looking expected return (e.g., your DCF IRR)
5) Prints intermediate values so you can see each step

USAGE:
- Put a CSV named like: nike.csv with columns: Date, Close
- Date format can be YYYY-MM-DD or any pandas-readable date
- Run script, edit parameters at top (expected_return, risk_free, freq)

If you have multiple stocks, run this per file or loop over a dict of tickers->csv.
"""

import pandas as pd
import numpy as np

# -----------------------------
# 0) INPUTS TO CONTROL
# -----------------------------
#Choose from nike, lulu, or atz_cn
STOCK = "atz_cn"

CSV_PATH = f"./data/{STOCK}.csv"

# Forward-looking expected annual return from your valuation (DCF IRR), as a decimal
# Example: 0.14 means 14% expected annual return.
expected_return_annual = 0.1297 if STOCK == "lulu" else 0.1494 if STOCK == "atz_cn" else 0.1424

# Annual risk-free rate us - taken from Bloomberg USGG12M Index, cad - taken from Bloomberg GCAN12M Index 
risk_free_annual = 0.03507 if STOCK == "nike" or STOCK == "lulu" else 0.02830

# Target return for Sortino:
# Common choice: risk-free rate (consistent) OR 0.0 (clean).
target_return_annual = risk_free_annual


# Return frequency for volatility calculations: "W" weekly (recommended) or "M" monthly
freq = "W"

# Annualization factor for frequency
# Weekly ~ 52, Monthly ~ 12
ann_factor = 52 if freq == "W" else 12


# -----------------------------
# 1) LOAD PRICE DATA
# -----------------------------
df = pd.read_csv(CSV_PATH)

# Ensure correct types and ordering
df["Date"] = pd.to_datetime(df["Date"])
df = df.sort_values("Date").reset_index(drop=True)

# Basic sanity check: need Close
if "Close" not in df.columns:
    raise ValueError("CSV must contain a 'Close' column.")

print("\nSTEP 1: Loaded price data")
print(df.head())
print(f"Expected Return for {STOCK}: {expected_return_annual}")


STEP 1: Loaded price data
        Date  Close
0 2020-01-03  19.23
1 2020-01-10  23.16
2 2020-01-17  24.71
3 2020-01-24  25.03
4 2020-01-31  25.03
Expected Return for atz_cn: 0.1494


In [62]:
# -----------------------------
# 2) RESAMPLE TO WEEKLY OR MONTHLY CLOSE
# -----------------------------
# Why: Daily returns are noisy; weekly/monthly are more stable for a pitch.
# We take the last available close in each week/month.
df = df.set_index("Date")
prices = df["Close"].resample(freq).last().dropna()

print(f"\nSTEP 2: Resampled prices to {freq} frequency (last close each period) for {STOCK}")
print(prices.head())


STEP 2: Resampled prices to W frequency (last close each period) for atz_cn
Date
2020-01-05    19.23
2020-01-12    23.16
2020-01-19    24.71
2020-01-26    25.03
2020-02-02    25.03
Freq: W-SUN, Name: Close, dtype: float64


In [63]:

# -----------------------------
# 3) COMPUTE PERIODIC RETURNS
# -----------------------------
# Return_t = (P_t / P_{t-1}) - 1
returns = prices.pct_change().dropna()

print(f"\nSTEP 3: Computed periodic returns (first few) for {STOCK}")
print(returns.head())

# (Optional) Inspect basic return stats
print(f"\nReturn stats (periodic) for {STOCK}:")
print(returns.describe())



STEP 3: Computed periodic returns (first few) for atz_cn
Date
2020-01-12    0.204368
2020-01-19    0.066926
2020-01-26    0.012950
2020-02-02    0.000000
2020-02-09    0.007191
Freq: W-SUN, Name: Close, dtype: float64

Return stats (periodic) for atz_cn:
count    311.000000
mean       0.008263
std        0.070143
min       -0.319828
25%       -0.030107
50%        0.005467
75%        0.040729
max        0.395200
Name: Close, dtype: float64


In [64]:
# -----------------------------
# 4) COMPUTE TOTAL VOLATILITY (PERIODIC + ANNUALIZED)
# -----------------------------
# Periodic volatility = std dev of periodic returns
vol_periodic = returns.std(ddof=1)

# Annualized volatility = periodic_vol * sqrt(annualization_factor)
vol_annual = vol_periodic * np.sqrt(ann_factor)

print(f"\nSTEP 4: Total volatility for {STOCK}")
print(f"Periodic volatility: {vol_periodic:.6f}")
print(f"Annualized volatility: {vol_annual:.6f}")


STEP 4: Total volatility for atz_cn
Periodic volatility: 0.070143
Annualized volatility: 0.505811


In [65]:
# -----------------------------
# 5) SHARPE RATIO
# -----------------------------
# Sharpe = (E[R] - Rf) / sigma
# Here, E[R] is your DCF IRR (expected_return_annual)
excess_return = expected_return_annual - risk_free_annual

if vol_annual == 0:
    raise ZeroDivisionError("Annualized volatility is zero; cannot compute Sharpe.")

sharpe = excess_return / vol_annual

print(f"\nSTEP 5: Sharpe ratio for {STOCK}")
print(f"Expected annual return: {expected_return_annual:.4f}")
print(f"Risk-free rate: {risk_free_annual:.4f}")
print(f"Excess return: {excess_return:.4f}")
print(f"Sharpe: {sharpe:.4f}")


STEP 5: Sharpe ratio for atz_cn
Expected annual return: 0.1494
Risk-free rate: 0.0283
Excess return: 0.1211
Sharpe: 0.2394


In [66]:
# -----------------------------
# 6) DOWNSIDE DEVIATION FOR SORTINO
# -----------------------------
# Sortino uses downside risk only:
# downside deviation = std dev of returns below a threshold (target)
#
# Important: target is annual; returns here are periodic.
# Convert annual target to periodic target:
# target_periodic = (1 + target_annual)^(1/ann_factor) - 1
target_periodic = (1.0 + target_return_annual) ** (1.0 / ann_factor) - 1.0

# "Downside returns" are negative deviations below target:
# downside = min(0, r_t - target_periodic)
downside = np.minimum(0.0, returns - target_periodic)

# Downside deviation (periodic) = sqrt(mean(downside^2))
downside_dev_periodic = np.sqrt(np.mean(downside ** 2))

# Annualize
downside_dev_annual = downside_dev_periodic * np.sqrt(ann_factor)

print(f"\nSTEP 6: Downside deviation (for Sortino) for {STOCK}")
print(f"Target annual return: {target_return_annual:.4f}")
print(f"Target periodic return: {target_periodic:.6f}")
print(f"Downside deviation (periodic): {downside_dev_periodic:.6f}")
print(f"Downside deviation (annualized): {downside_dev_annual:.6f}")


STEP 6: Downside deviation (for Sortino) for atz_cn
Target annual return: 0.0283
Target periodic return: 0.000537
Downside deviation (periodic): 0.043585
Downside deviation (annualized): 0.314293


In [67]:
# -----------------------------
# 7) SORTINO RATIO
# -----------------------------
# Sortino = (E[R] - target) / downside_deviation
sortino_numerator = expected_return_annual - target_return_annual

if downside_dev_annual == 0:
    raise ZeroDivisionError("Downside deviation is zero; cannot compute Sortino.")

sortino = sortino_numerator / downside_dev_annual

print(f"\nSTEP 7: Sortino ratio for {STOCK}")
print(f"Numerator (E[R] - target): {sortino_numerator:.4f}")
print(f"Sortino: {sortino:.4f}")


# -----------------------------
# 8) FINAL OUTPUT SUMMARY
# -----------------------------
print(f"\nSUMMARY for {STOCK}")
print(f"Sharpe  : {sharpe:.4f}")
print(f"Sortino : {sortino:.4f}")
print(f"Vol (ann): {vol_annual:.4f}")
print(f"Downside dev (ann): {downside_dev_annual:.4f}")


STEP 7: Sortino ratio for atz_cn
Numerator (E[R] - target): 0.1211
Sortino: 0.3853

SUMMARY for atz_cn
Sharpe  : 0.2394
Sortino : 0.3853
Vol (ann): 0.5058
Downside dev (ann): 0.3143


In [68]:
# -----------------------------
# 9) SAVE RESULTS TO TXT
# -----------------------------
output_txt_path = f"./outputs/{STOCK}_risk_metrics.txt"

with open(output_txt_path, "w") as f:
    f.write(f"RISK-ADJUSTED RETURN METRICS for {STOCK}\n")
    f.write("----------------------------\n\n")

    f.write(f"Expected Return (DCF IRR): {expected_return_annual:.4f}\n")
    f.write(f"Risk-Free Rate (USD):      {risk_free_annual:.4f}\n\n")

    f.write(f"Annual Volatility:         {vol_annual:.4f}\n")
    f.write(f"Downside Deviation:        {downside_dev_annual:.4f}\n\n")

    f.write(f"Sharpe Ratio:              {sharpe:.4f}\n")
    f.write(f"Sortino Ratio:             {sortino:.4f}\n")

print(f"\nResults saved to {output_txt_path}")



Results saved to ./outputs/atz_cn_risk_metrics.txt
